# Statistiques sur les trains TGV de la SNCF

In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/regularite-mensuelle-tgv-aqst.csv', delimiter=";")
df.head()

,Date,Service,Gare de départ,Gare d'arrivée,Durée moyenne du trajet,Nombre de circulations prévues,Nombre de trains annulés,Commentaire annulations,Nombre de trains en retard au départ,Retard moyen des trains en retard au départ,...,Nombre trains en retard > 15min,Retard moyen trains en retard > 15 (si liaison concurrencée par vol),Nombre trains en retard > 30min,Nombre trains en retard > 60min,Prct retard pour causes externes,Prct retard pour cause infrastructure,Prct retard pour cause gestion trafic,Prct retard pour cause matériel roulant,Prct retard pour cause gestion en gare et réutilisation de matériel,"Prct retard pour cause prise en compte voyageurs (affluence, gestions PSH, correspondances)"
0,2018-01,National,ANGERS SAINT LAUD,PARIS MONTPARNASSE,91,393,2,NaN,204,6.400490,...,35,6.072847,19,4,37.500000,37.500000,8.928571,5.357143,8.928571,1.785714
1,2018-01,National,LA ROCHELLE VILLE,PARIS MONTPARNASSE,165,222,0,NaN,8,2.875000,...,22,5.696096,5,0,15.384615,30.769231,38.461538,11.538462,3.846154,0.000000
2,2018-01,National,PARIS MONTPARNASSE,BORDEAUX ST JEAN,143,775,7,NaN,148,9.377815,...,45,2.546962,21,4,21.666667,25.000000,11.666667,40.000000,1.666667,0.000000
3,2018-01,National,PARIS MONTPARNASSE,LAVAL,84,217,0,NaN,32,11.609375,...,16,5.822811,5,2,24.242424,54.545455,3.030303,12.121212,3.030303,3.030303
4,2018-01,National,PARIS MONTPARNASSE,NANTES,124,508,3,NaN,71,7.235211,...,39,5.292211,18,8,33.333333,22.222222,16.666667,20.370370,5.555556,1.851852
